# MEIRL

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
area = "london"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [3]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [4]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [5]:
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [6]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [7]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [8]:
def update_position(current_position, eps, reward):
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        updated_position = random.choice(value_list[value_list["value"] == max(value_list["value"])].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]
    else:
        updated_position = random.choice(value_list["value"].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]

    return updated_position

In [9]:
# how many iterations?
Y = []
iteration = 50
for i in range (iteration):
    epsilon = 1 / (i+1)  
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        start_position = random.choice(adjacency_matrix.columns)
        current_position = start_position
        # agent will walk for 50 steps
        for k in range(50):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*0.05) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 185.480769231
0.0885112102767
1 144.507548526
0.0476499807502
2 137.463399504
0.0937301005695
3 136.681396618
0.0425195514553
4 130.820186183
0.0619504562842
5 135.375624376
0.129978741136
6 139.326923077
0.0751393503015
7 146.115244245
0.0751673063498
8 155.745622264
0.0688316245962
9 141.336406744
0.129062512701
10 137.486538462
0.103758372286
11 147.894844517
0.0886570608881
12 125.548617113
0.156974323903
13 146.34508547
0.0793909646534
14 144.429487179
0.0787297431914
15 158.373538462
0.0584569958859
16 150.951309329
0.122459771939
17 148.580769231
0.196889802353
18 158.574175824
0.150184122709
19 140.371794872
0.16259885093
20 135.758144796
0.142340685415
21 127.325123707
0.218857424057
22 119.708842559
0.215649085109
23 124.996540497
0.252379028363
24 135.287962038
0.240338032097
25 110.525859806
0.260547724862
26 134.26486014
0.169130110037
27 114.290309036
0.282767194296
28 114.932692308
0.19433359911
29 107.865384615
0.261540592486
30 114.446348399
0.230869282981
31 110.604

# Local Depth

In [9]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [10]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [11]:
correlation_local_depth

0.45406804231592879

# Connectivity 

In [12]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.32460389310488996

# Global Depth

In [13]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.45406704994670616